Determining the distortion angle of (tetha1+tetha2)-(tetha3+tetha4)

In [ ]:
%matplotlib qt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns

def compute_degree(A, B, C):
    BA = A - B
    BC = C - B
    cos_theta = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    angle_rad = np.arccos(cos_theta)
    angle_deg = np.degrees(angle_rad)

    return angle_deg

# Setup
directory = Path.cwd().parent.parent / "porphyrin-angles"
fname = directory / "POR-Water-pos-1.xyz"

# Read file
with open(fname) as f:
    lines = f.readlines()

# Identify frame start lines
start_idxs = [i for i, line in enumerate(lines) if line.strip().startswith("i")]
start_idxs.append(len(lines))  # Append end marker

# Map atom labels to line indices (from visual inspection)
atom_indices = {
    "Fe": 768,
    "N1": 769, "N2": 770, "N3": 771, "N4": 772,
    "S": 773, "H": 774,
    "C1": 775, "C2": 776, "C3": 777, "C4": 779, "C5": 780,
    "C6": 781, "C7": 784, "C8": 785, "C9": 787, "C10": 788,
    "C11": 789, "C12": 792, "C13": 793, "C14": 795, "C15": 796,
    "C16": 797, "C17": 800, "C18": 801, "C19": 803, "C20": 804,
    "O1": 807, "O2": 808
}

# Define which angles to compute: (A, B, C) means angle at B between A-B-C
angle_defs = {
    "O1-C19-Fe": ("O1", "C19", "Fe"),
    "O1-C10-Fe": ("O1", "C10", "Fe"),
    "O1-C4-Fe": ("O1", "C4", "Fe"),
    "O1-C14-Fe": ("O1", "C14", "Fe")
}

# Store angles
all_angles = {name: [] for name in angle_defs}

# Process each frame
for st, en in zip(start_idxs[:-1], start_idxs[1:]):
    frame_lines = lines[st+1:en]  # Skip the "i" line
    coords = {}

    # Extract only needed atoms
    for atom, idx in atom_indices.items():
        line = frame_lines[idx]
        coords[atom] = np.array(line.strip().split()[1:], dtype=float)

    # Compute all requested angles
    for angle_name, (a, b, c) in angle_defs.items():
        angle = compute_degree(coords[a], coords[b], coords[c])
        all_angles[angle_name].append(angle)

# Convert to long-format DataFrame
angle_df = pd.DataFrame(all_angles)
angle_df["frame"] = range(len(angle_df))
long_df = angle_df.melt(id_vars="frame", var_name="angle_type", value_name="angle_deg")


In [ ]:
angle_order = ['O1-C19-Fe', 'O1-C10-Fe', 'O1-C4-Fe', 'O1-C14-Fe']
def compute_difference(group):
    ordered = group.set_index('angle_type').loc[angle_order, 'angle_deg']
    return (ordered.iloc[0] + ordered.iloc[1]) - (ordered.iloc[2] + ordered.iloc[3])

result = long_df.groupby('frame').apply(compute_difference).reset_index(name='angle_diff')

sns.histplot(result, x="angle_diff", stat="probability", color="seagreen")

In [ ]:
## for carbons
df_sub = long_df[long_df["angle_type"].str.contains("S-Fe-C")]
df_sub["bond"] = df_sub["angle_type"].str.extract(r"(C\d+)")
df_sub["bond"] = 'S-Fe-' + df_sub["bond"]
g = sns.FacetGrid(data=df_sub, col="bond", col_wrap=4)
g.map_dataframe(sns.histplot, bins=50, x="angle_deg", stat="probability")

stats = df_sub.groupby("bond")["angle_deg"].agg(["min", "max", "mean"]).round(2)
for ax, carbon in zip(g.axes.flat, stats.index):
    row = stats.loc[carbon]
    text = f"min: {row['min']}°\nmax: {row['max']}°\nmean: {row['mean']}°"
    ax.legend([text], loc="upper right", frameon=False, fontsize=8)

In [ ]:
# for rest
df_sub = long_df[~long_df["angle_type"].str.contains("C")]
g = sns.FacetGrid(data=df_sub, col="angle_type", sharex=False)
g.map_dataframe(sns.histplot, bins=50, x="angle_deg", stat="probability")

stats = df_sub.groupby("angle_type")["angle_deg"].agg(["min", "max", "mean"]).round(2)
for ax, carbon in zip(g.axes.flat, stats.index):
    row = stats.loc[carbon]
    text = f"min: {row['min']}°\nmax: {row['max']}°\nmean: {row['mean']}°"
    ax.legend([text], loc="upper right", frameon=False, fontsize=8)